In [52]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


In [53]:
dataMRI = pd.read_csv('mri.csv')
dataCSF = pd.read_csv('csf.csv')
dataUDS = pd.read_csv('uds.csv')

In [54]:
merged_df = pd.merge(pd.merge(dataUDS, dataCSF, on='NACCID', how='outer'), dataMRI, on='NACCID', how='outer')

In [55]:
merged_df = merged_df[['NACCID','SEX', 'EDUC', 'NACCAGE', 'VEG', 'ANIMALS', 'TRAILA', 'TRAILB', 'CRAFTDRE', 'MINTTOTS', 'DIGBACCT', 'MEMPROB', 'DROPACT', 'WRTHLESS', 'BETTER', 'BORED', 'HELPLESS', 'TAXES', 'BILLS', 'REMDATES', 'TRAVEL', 'NACCUDSD']]

In [56]:
merged_df = merged_df.dropna(axis=0, how='any')

In [57]:
merged_df = merged_df[['SEX', 'EDUC', 'NACCAGE', 'VEG', 'ANIMALS', 'TRAILA', 'TRAILB', 'CRAFTDRE', 'MINTTOTS', 'DIGBACCT', 'MEMPROB', 'DROPACT', 'WRTHLESS', 'BETTER', 'BORED', 'HELPLESS', 'TAXES', 'BILLS', 'REMDATES', 'TRAVEL', 'NACCUDSD']]

In [63]:
merged_df = merged_df[(merged_df['NACCUDSD'] != 1) & (merged_df['NACCUDSD'] != 2)]

In [60]:
def transform_value(x):
    if x == 3:
        return 0
    elif x == 4:
        return 1
    else:
        return x


In [66]:
merged_df['NACCUDSD'] = merged_df['NACCUDSD'].apply(transform_value)

In [67]:
x = merged_df.iloc[:, :-1].values
y = merged_df.iloc[:, -1].values

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y) 

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scalerFunction = StandardScaler()
X_train = scalerFunction.fit_transform(X_train)
X_test = scalerFunction.transform(X_test)

In [68]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='tanh', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(2, activation='sigmoid') 
])

c:\Users\wesle\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [69]:
model.compile(optimizer='RMSProp',
              loss='binary_crossentropy',  
              metrics=['accuracy'])

In [70]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7787 - loss: 0.5380 - val_accuracy: 0.8211 - val_loss: 0.4116
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8544 - loss: 0.3532 - val_accuracy: 0.8252 - val_loss: 0.3957
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8435 - loss: 0.3579 - val_accuracy: 0.8252 - val_loss: 0.3935
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8641 - loss: 0.3377 - val_accuracy: 0.8232 - val_loss: 0.3940
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8572 - loss: 0.3363 - val_accuracy: 0.8211 - val_loss: 0.3946
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8541 - loss: 0.3487 - val_accuracy: 0.8232 - val_loss: 0.3944
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8568 - loss: 0.3225 - val_accuracy: 0.8191 - val_loss: 0.3952
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8621 - loss: 0.3285 - val_accuracy: 0.8171 - val_loss:

In [71]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - accuracy: 0.8190 - loss: 0.4248
Test loss: 0.4132380187511444
Test accuracy: 0.8211382031440735


In [72]:
#measuring performance with AUC

model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(name='auc')])



In [73]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - auc: 0.9436 - loss: 0.3036 - val_auc: 0.9022 - val_loss: 0.3990
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - auc: 0.9422 - loss: 0.3086 - val_auc: 0.9022 - val_loss: 0.3989
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc: 0.9445 - loss: 0.3032 - val_auc: 0.9024 - val_loss: 0.3997
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - auc: 0.9428 - loss: 0.3061 - val_auc: 0.9013 - val_loss: 0.4004
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - auc: 0.9461 - loss: 0.3003 - val_auc: 0.9018 - val_loss: 0.3996
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - auc: 0.9459 - loss: 0.2982 - val_auc: 0.9020 - val_loss: 0.3990
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc: 0.9386 - loss: 0.3164 - val_auc: 0.9021 - val_loss: 0.4004
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc: 0.9502 - loss: 0.2879 - val_auc: 0.9017 - val_loss: 0.4001
Epoch 9/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - auc: 0.9520 - loss: 

In [74]:

# Evaluate the model
results = model.evaluate(X_test, y_test)
print("Test Loss, Test AUC:", results)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - auc: 0.8929 - loss: 0.4386 
Test Loss, Test AUC: [0.4516185224056244, 0.8879622220993042]
